In [5]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open("../../settings.ini"))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

In [6]:
local_path = "../data_final/"
remote_path = "/"

In [7]:
# from local file postgres.py
import postgres
# from local file commons.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file

import json
import pandas as pd
from sqlalchemy import create_engine

In [8]:
df1 = download_file(remote_path, local_path, "housing_census.csv")
list(df1)

Already downloaded. Using: ../data_final/housing_census.csv


['area_id',
 'district_id',
 'housing_type',
 'housing_occupation',
 'walls_material',
 'ceiling_material',
 'floor_material',
 'rooms',
 'water_source',
 'cant_viv',
 'cant_hog',
 'cant_per',
 'comuna_datachile_id']

In [9]:
df1.head()

,area_id,district_id,housing_type,housing_occupation,walls_material,ceiling_material,floor_material,rooms,water_source,cant_viv,cant_hog,cant_per,comuna_datachile_id
0,1,1,1,1,1,1,1,1,1,1,1,2,113
1,1,1,1,1,1,1,1,2,1,7,7,19,113
2,1,1,1,1,1,1,1,3,1,3,3,13,113
3,1,1,1,1,1,1,1,4,1,1,1,6,113
4,1,1,1,1,1,1,1,5,1,1,1,3,113


In [10]:
df1 = df1.rename(columns = {"comuna_datachile_id": "comuna_id"})

In [11]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df1, "census", "fact_housing_census")

DROP TABLE IF EXISTS census.fact_housing_census;
CREATE TABLE "census"."fact_housing_census" (
"area_id" INTEGER,
  "district_id" INTEGER,
  "housing_type" INTEGER,
  "housing_occupation" INTEGER,
  "walls_material" INTEGER,
  "ceiling_material" INTEGER,
  "floor_material" INTEGER,
  "rooms" INTEGER,
  "water_source" INTEGER,
  "cant_viv" INTEGER,
  "cant_hog" INTEGER,
  "cant_per" INTEGER,
  "comuna_id" INTEGER
)
COPY "census"."fact_housing_census" ("area_id","district_id","housing_type","housing_occupation","walls_material","ceiling_material","floor_material","rooms","water_source","cant_viv","cant_hog","cant_per","comuna_id") FROM STDIN WITH CSV HEADER DELIMITER ',';


In [12]:
engine.execute("""
CREATE INDEX fact_housing_census_index 
ON census.fact_housing_census (comuna_id)
""")

In [13]:
# Read questionnaire file
questionnaire = pd.ExcelFile("https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4xZxDpyDY4NursNbrsWlCqjREdmBbfC1EMlz4UGQe7M8wMA7Mqw8tZbAcBdkjgBzloyQdcnhiv10C/pub?output=xlsx")
labels = pd.read_excel(questionnaire, "Labels")

# Questions related with this ingest
questions = ["P01", "P02", "P03A", "P03B", "P03C", "P04", "P05"]

for q in questions:
    df = pd.read_excel(questionnaire, q)
    label = labels[labels["id"] == q].iloc[0]["label"]
    print (inline_dimension_xml(df, label, "id", "name_es", label))


<Dimension name="Housing Type" foreignKey="housing_type">
  <Hierarchy hasAll="true">
            
<InlineTable alias="housing_type">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Casa</Value>
      <Value column="es_description">Casa</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Departamento en edificio</Value>
      <Value column="es_description">Departamento en edificio</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Vivienda tradicional indígena (ruka, pae pae u otras)</Value>
      <Value column="es_description">Vivienda tradicional indígena (ruka, pae pae u otras)</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Pieza e

KeyError: 'name_es'